In [0]:
# find duplicate records
# remove duplicate records


data =[(1,'abc@gmail.com'),(2,'def@gmail.com'),(1,'abc@gmail.com')]
column=['id','name']

df =spark.createDataFrame(data,column)
df.show()

+---+-------------+
| id|         name|
+---+-------------+
|  1|abc@gmail.com|
|  2|def@gmail.com|
|  1|abc@gmail.com|
+---+-------------+



In [0]:
from pyspark.sql.functions import count, col

display(df.groupBy("id","name").count().filter(col("count")>1))



id,name,count
1,abc@gmail.com,2


In [0]:
df.createOrReplaceTempView("myDataset")



In [0]:
%sql
select id, name, count(*)
from myDataset
group by 1,2
having count(*)>1

id,name,count(1)
1,abc@gmail.com,2


In [0]:
df.distinct().show()

+---+-------------+
| id|         name|
+---+-------------+
|  1|abc@gmail.com|
|  2|def@gmail.com|
+---+-------------+



Q2

In [0]:
customer_data=[(1,'Manish'),(2,'Rahul'),(3,'Monu'),(4,'Ram')]
schema=["Customer_ID", "Customer_Name"]

order_data=[(1,4),(3,2)]
schema1=["Order_ID", "Customer_ID"]

In [0]:
customer_df = spark.createDataFrame(customer_data, schema)
order_df = spark.createDataFrame(order_data, schema1)

customer_df.show()

order_df.show()

+-----------+-------------+
|Customer_ID|Customer_Name|
+-----------+-------------+
|          1|       Manish|
|          2|        Rahul|
|          3|         Monu|
|          4|          Ram|
+-----------+-------------+

+--------+-----------+
|Order_ID|Customer_ID|
+--------+-----------+
|       1|          4|
|       3|          2|
+--------+-----------+



In [0]:
df_joined = customer_df.join(order_df, customer_df.Customer_ID==order_df.Customer_ID,"left").select(customer_df.Customer_ID, customer_df.Customer_Name).filter(order_df.Order_ID.isNull())

df_joined.show()

+-----------+-------------+
|Customer_ID|Customer_Name|
+-----------+-------------+
|          1|       Manish|
|          3|         Monu|
+-----------+-------------+



In [0]:
df_joined = customer_df.join(order_df, customer_df.Customer_ID==order_df.Customer_ID,"inner").select(customer_df.Customer_ID, customer_df.Customer_Name)

df_joined.show()

+-----------+-------------+
|Customer_ID|Customer_Name|
+-----------+-------------+
|          2|        Rahul|
|          4|          Ram|
+-----------+-------------+



Q3

In [0]:
emp_data=[('Manish' , 1 , 75000),
('Raghav' , 1 , 85000 ),
('surya' , 1 , 80000 ),
('virat' , 2 , 70000),
('rohit' , 2 , 75000),
('jadeja' , 3 , 85000),
('anil' , 3 , 55000),
('sachin' , 3 , 55000),  
('zahir', 4, 60000),
('bumrah' , 4 , 65000) ]
schema= ["emp_name" ,"dept_id" ,"salary"]

dept_data = [(1, 'DATA ENGINEER'),(2, 'SALES'),(3, 'SOFTWARE'),(4, 'HR')]
schema1=['dept_id','dept_name']

In [0]:
emp_df = spark.createDataFrame(emp_data, schema)
emp_df.show(5)


dept_df = spark.createDataFrame(dept_data, schema1)
dept_df.show(5)


+--------+-------+------+
|emp_name|dept_id|salary|
+--------+-------+------+
|  Manish|      1| 75000|
|  Raghav|      1| 85000|
|   surya|      1| 80000|
|   virat|      2| 70000|
|   rohit|      2| 75000|
+--------+-------+------+
only showing top 5 rows

+-------+-------------+
|dept_id|    dept_name|
+-------+-------------+
|      1|DATA ENGINEER|
|      2|        SALES|
|      3|     SOFTWARE|
|      4|           HR|
+-------+-------------+



In [0]:
from pyspark.sql.functions import max

In [0]:
df_joined = emp_df.join(dept_df, emp_df.dept_id==dept_df.dept_id, "left")
                
df_joined_res = df_joined.groupBy("dept_name").agg(max("salary").alias("max_sal"))


df_joined_res.show()

+-------------+-------+
|    dept_name|max_sal|
+-------------+-------+
|DATA ENGINEER|  85000|
|        SALES|  75000|
|     SOFTWARE|  85000|
|           HR|  65000|
+-------------+-------+



In [0]:
df_res = df_joined.join(df_joined_res, df_joined.dept_name==df_joined_res.dept_name, "left")\
                    .filter(col("salary")==col("max_sal"))

df_res.show()

+--------+-------+------+-------+-------------+-------------+-------+
|emp_name|dept_id|salary|dept_id|    dept_name|    dept_name|max_sal|
+--------+-------+------+-------+-------------+-------------+-------+
|  Raghav|      1| 85000|      1|DATA ENGINEER|DATA ENGINEER|  85000|
|   rohit|      2| 75000|      2|        SALES|        SALES|  75000|
|  jadeja|      3| 85000|      3|     SOFTWARE|     SOFTWARE|  85000|
|  bumrah|      4| 65000|      4|           HR|           HR|  65000|
+--------+-------+------+-------+-------------+-------------+-------+

